In [20]:
import math # Pour les fonctions log, exp...
from scipy.stats import norm # Pour les fonctions de répartition et de densité de lois normales
from statistics import stdev
import calendar
import copy
import datetime
import bql
bq = bql.Service()
from scipy.interpolate import interp1d, Rbf, interp2d, RectBivariateSpline
import numpy as np
import pandas as pd
import ipywidgets as widgets
import bqviz as bqv
import bqplot as bqp
from ipydatagrid import DataGrid, TextRenderer
from collections import OrderedDict
from bqwidgets import TickerAutoComplete
from IPython.display import display
import re
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
from IPython.display import FileLink

from tqdm import tqdm
# from tqdm.notebook import tqdm

import traceback, logging # For debugging

import io
import os
# from __future__ import absolute_import
import sys
# sys.path.append('../work/')

In [21]:
import scipy.optimize as sciopt

In [22]:
# file = open("disclaimer_vol (1).png", "rb")
# image = file.read()
# widgets.Image(
#     value=image,
#     format='png',
#     width=1000,
#     height=400,
# )

In [23]:
# file = open("disclaimer_forward.png", "rb")
# image = file.read()
# widgets.Image(
#     value=image,
#     format='png',
#     width=1000,
#     height=400,
# )

# SVI calibrator and local volatility model vanilla pricing assessor

This Bloomberg BQuant app implements a local volatility model, and assesses its quality in re-pricing vanilla options. 

So far, the following underlying assets have been implemented (other equity assets can easily be included) : 
- SX5E Index
- SPX Index

This app uses the underlying asset implied volatility surface obtained :
- either from Super Derivatives (1st tab) : in that case, you have to upload it with the relevant widget, and remind the valuation date.
- either from Bloomberg directly (2nd tab) : in that case just choose the underlying and the valuation date, and the data will be fetched automatically.

This implied volatility surface will then be smoothed, using an SVI procedure. After smoothing, the local volatility surface will be calculated.

Finally, choose the number of Monte Carlo paths you want to simulate, and assess (in bps) the pricing error of your local volatility model for vanilla options. 

In [24]:
def getSVICurve(parameters, log_moneyness): # parameters doit être de la forme (a, b, rho, m, sigma)
    a = parameters[0]
    b = parameters[1]
    rho = parameters[2]
    m = parameters[3]
    sigma = parameters[4]
    return([a+b*(rho*(k-m)+np.sqrt((k-m)**2 + sigma**2)) for k in log_moneyness])

def butterflyArbitrage(volCoef, S, interest, dividend, time2Maturity, vecStrikePrice):    
    a = volCoef[0]
    b = volCoef[1]
    rho = volCoef[2]
    m = volCoef[3]
    tau = volCoef[4]
    g = gFunction(a, b, rho, m, tau, S, interest, dividend, time2Maturity, vecStrikePrice)
    return np.exp([0 if gi>=0 else 1 for gi in g])

# def envelopeCondition(pAdjusted, bid, ask):
#     return np.exp(np.clip([0 if (pi>=bi and pi<=ai) else 2*np.fabs(pi-(ai+bi)/2)/(ai-bi) for pi, bi, ai in zip(pAdjusted, bid, ask)],
#                           a_min=None,
#                           a_max=10))


# On supporse que la condition d'enveloppe est vérifiée > simplification car les données sur prix bid et ask sont sales via Bloomberg
def globalOptimization(X, volMarket, optionType, S0, interest, dividend, time2Maturity, vecStrikePrice):
    moneyness = [np.log(k / S0) - (interest - dividend) *  time2Maturity for k in vecStrikePrice]
    volAdjusted = getSVICurve(X, moneyness)    
    
    ## penalizations
    consButterfly = butterflyArbitrage(X, S0, interest, dividend, time2Maturity, vecStrikePrice)
    # consEnvelope = envelopeCondition(pAdjusted, bid, ask)

    OF = np.array([((volAdj/volMar - 1)**2) * bi for volAdj,volMar, bi in zip(volAdjusted, volMarket, consButterfly)])
    return np.sum(OF)

def callbackFunction(X, convergence):
    xProgress.append(X)

def gFunction(a, b, rho, m, tau, S, interest, dividend, time2Maturity, vecStrike):
    moneyness = [np.log(k / S) - (interest - dividend) *  time2Maturity for k in vecStrike]
    g = []
    for k in moneyness:
        w = (SVIFunction(a, b, rho, m, tau, k)**2) * time2Maturity
        wd = SVIFunctionDerivative(b, rho, m, tau, k) * time2Maturity
        wdd = SVIFunction2ndDerivative(b, m, tau, k) * time2Maturity
        if w!=0:
            g.append((1 - k * wd / (2 * w)) ** 2 - (wd * wd / 4) * (1 / w + 0.25) + wdd / 2)
        else:
            g.append(-1)
    return g

In [25]:
def SVIFunction(a, b, rho, m, tau, k):
    y = a + b * (rho * (k - m) + np.sqrt((k - m) ** 2 + tau ** 2))
    if y>=0:
        return np.sqrt(y) # it returns the square root of the variance
    else:
        return 0
    
def SVIFunctionDerivative(b, rho, m, tau, k): # dérivée par rapport à k
    return b * (rho + (k - m) / np.sqrt((k - m) ** 2 + tau ** 2))

def SVIFunction2ndDerivative(b, m, tau, k): # dérivée seconde
    return b * tau * tau / np.power((k - m) ** 2 + tau ** 2, 1.5)

def gFunction(a, b, rho, m, tau, S, interest, dividend, time2Maturity, vecStrike):
    moneyness = [np.log(k / S) - (interest - dividend) *  time2Maturity for k in vecStrike]
    g = []
    for k in moneyness:
        w = (SVIFunction(a, b, rho, m, tau, k)**2) * time2Maturity
        wd = SVIFunctionDerivative(b, rho, m, tau, k) * time2Maturity
        wdd = SVIFunction2ndDerivative(b, m, tau, k) * time2Maturity
        if w!=0:
            g.append((1 - k * wd / (2 * w)) ** 2 - (wd * wd / 4) * (1 / w + 0.25) + wdd / 2)
        else:
            g.append(-1)
    return g

In [26]:
underlying_box = TickerAutoComplete(description='Underlying Asset',style={'description_width':'initial'},yellow_keys=['Index'])

upload_implied_vol_button = widgets.FileUpload(
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    description = 'Upload implied volatility',
    style={'description_width':'initial'},
    layout = widgets.Layout(width = '310px'),
    multiple=False  # True to accept multiple files upload else False
)

upload_forward_button = widgets.FileUpload(
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    description = 'Upload rates and dividends',
    style={'description_width':'initial'},
    layout = widgets.Layout(width = '310px'),
    multiple=False  # True to accept multiple files upload else False
)

valuation_date = widgets.DatePicker(
    description='Valuation Date', 
    style={'description_width':'initial'},
    disabled=False
)

number_MC_paths = widgets.BoundedIntText(
    value=5000,
    min=50,
    max=100000,
    step=50,
    description='Number of paths to simulate by expiries :',
    style={'description_width':'initial'},
    disabled=False
)

html_out = widgets.Output()

vol_file_name = widgets.Label()
vol_file_name.layout.width = '400px'
forward_file_name = widgets.Label()
forward_file_name.layout.width = '400px'
status_label = widgets.Label()
status_label.layout.width = '400px'
status_label2 = widgets.Label()
status_label2.layout.width = '400px'

button_run1 = widgets.Button(description='Reset', button_style='Danger')
button_run2 = widgets.Button(description='Run SVI on SuperDerivatives Data', button_style='Success', layout = widgets.Layout(width='250px'))
button_run3 = widgets.Button(description='Remove file')
button_run4 = widgets.Button(description='Remove file')
button_run5 = widgets.Button(description='Show file name')
button_run6 = widgets.Button(description='Show file name')
button_run7 = widgets.Button(description='Run SVI on Bloomberg Data', button_style='Success', layout = widgets.Layout(width='200px'))
button_run8 = widgets.Button(description='Assess Model', button_style='Success', layout = widgets.Layout(width='200px'))

In [27]:
def LocalVolatilitySurfaceSuperDerivatives(InterestCurve, VolatilitySurface, DividendCurve):
    
    spot = VolatilitySurface['SPOT'][0] # S_0
    strikes_to_interpolate = list(VolatilitySurface.pivot_table(columns='Strike', index='Expiry', values='Volatility').columns)
    expiries_to_interpolate = list(VolatilitySurface.pivot_table(columns='Strike', index='Expiry', values='Volatility').index)

    ivol_matrix = (VolatilitySurface.pivot_table(columns='Strike', index='Expiry', values='Volatility').values) / 100
    
    dvd_interpolation = interp1d(DividendCurve['TIME'] / 365, DividendCurve['DIVIDEND'], fill_value = 'extrapolate')
    rate_interpolation = interp1d(InterestCurve['TIME'] / 365, InterestCurve['RATE'], fill_value = 'extrapolate')

    mons_to_interpolate = [x/spot for x in strikes_to_interpolate]

    loc_vol_df = pd.DataFrame({})

    for i in tqdm(range(1, len(mons_to_interpolate) - 1)):
        for j in range(1, len(expiries_to_interpolate)):

            ivol_t_k = ivol_matrix[j][i]
            ivol_tminus_k = ivol_matrix[j-1][i]
            ivol_t_kminus = ivol_matrix[j][i-1]
            ivol_t_kplus = ivol_matrix[j][i+1]

            rate_t = float(rate_interpolation(expiries_to_interpolate[j]))
            dvd_t = float(dvd_interpolation(expiries_to_interpolate[j]))

            d1 = (math.log(1/mons_to_interpolate[i]) + (rate_t - dvd_t + (ivol_t_k**2) / 2) * expiries_to_interpolate[j]) / (ivol_t_k * math.sqrt(expiries_to_interpolate[j]))

            dsigma_dt = (ivol_t_k - ivol_tminus_k) / ((expiries_to_interpolate[j] - expiries_to_interpolate[j-1]))
            dsigma_dk = (ivol_t_k - ivol_t_kminus) / ((mons_to_interpolate[i] - mons_to_interpolate[i-1]))
            d2sigma_dk2 = (ivol_t_kplus - 2 * ivol_t_k + ivol_t_kminus) / ((mons_to_interpolate[i] - mons_to_interpolate[i-1])**2)

            numerator = ivol_t_k ** 2 + 2 * ivol_t_k * expiries_to_interpolate[j] * dsigma_dt + 2 * (rate_t - dvd_t) * ivol_t_k * mons_to_interpolate[i] * expiries_to_interpolate[j] * dsigma_dk
            denominator = ((1 + mons_to_interpolate[i] * d1 * dsigma_dk * math.sqrt(expiries_to_interpolate[j])) ** 2) + ivol_t_k * (mons_to_interpolate[i]**2) * expiries_to_interpolate[j] * (d2sigma_dk2 - d1 * (dsigma_dk**2) * math.sqrt(expiries_to_interpolate[j]))

            loc_vol = math.sqrt(max(0,numerator / denominator))

            if loc_vol == 0:
                loc_vol = (ivol_t_k)
            # print(loc_vol)

            data = pd.DataFrame({'Expiry' : [expiries_to_interpolate[j] * 365], 'Moneyness' : [mons_to_interpolate[i] * 100], 'Local Volatility' : [loc_vol * 100], 'Expiry in years' : [expiries_to_interpolate[j]], 'Strike' : [mons_to_interpolate[i] * spot]})

            loc_vol_df = loc_vol_df.append(data, ignore_index = True)

    loc_vol_df = loc_vol_df.rename(columns = {'Local Volatility' : 'Volatility'})
    
    return(loc_vol_df)

# def CorrectLocalVolatilitySurfaceSuperDerivatives(LocalVolatilitySurface):
    
#     bottom_threshold = np.percentile(LocalVolatilitySurface['Volatility'], 5) - 2.5
#     top_threshold = np.percentile(LocalVolatilitySurface['Volatility'], 95) + 2.5
    
#     print(bottom_threshold, top_threshold)

#     unproblematic_loc_vols = LocalVolatilitySurface[LocalVolatilitySurface['Volatility'] > bottom_threshold]
#     unproblematic_loc_vols = unproblematic_loc_vols[unproblematic_loc_vols['Volatility'] < top_threshold]
#     problematic_loc_vols = LocalVolatilitySurface[LocalVolatilitySurface['Volatility'] <= bottom_threshold].append(LocalVolatilitySurface[LocalVolatilitySurface['Volatility'] >= top_threshold])

#     for index in problematic_loc_vols.index:
#         correct_loc_vol = interp1d(unproblematic_loc_vols[unproblematic_loc_vols['Moneyness'] == problematic_loc_vols['Moneyness'][index]]['Expiry'], unproblematic_loc_vols[unproblematic_loc_vols['Moneyness'] == problematic_loc_vols['Moneyness'][index]]['Volatility'], fill_value = 'extrapolate')
#         problematic_loc_vols['Volatility'][index] = correct_loc_vol(problematic_loc_vols['Expiry'][index])

#     corrected_loc_vol = problematic_loc_vols.append(unproblematic_loc_vols).sort_index()
    
#     return(corrected_loc_vol)

def runSVI_SuperDerivatives(_):
    
    # try:
        
        global VolMaxime, SmoothImpliedVolSurfFinal, corrected_loc_vol, InterestCrv, DividendCrv, spot
        
        button_run2.disabled = True
    
        status_label.value = 'Initialising (...)'
        
        UnderlyingTicker = underlying_box.value

        ValuationDate = valuation_date.value
        price = bq.data.px_last(dates = bq.func.range(ValuationDate,ValuationDate))
        item1 = {'PX_LAST': price}
        req = bql.Request(UnderlyingTicker, item1)
        res = bq.execute(req)
        df = bql.combined_df(res)
        df = df.reset_index()
        spot = df['PX_LAST'][0]

        VolMaximeRaw = pd.read_csv(io.BytesIO(upload_implied_vol_button.value[list(upload_implied_vol_button.value.keys())[0]]['content']), skiprows=[0,1])
        
        expiries_maxime = [0.05, 0.1, 0.25, 0.375, 0.5, 0.75, 1, 1.5, 2, 3, 4, 5, 6, 7, 8, 9, 10]

        VolMaximeRaw = VolMaximeRaw.set_index('Term')
        VolMaximeRaw = VolMaximeRaw.drop(['Strike'], axis = 1) 
        VolMaximeRawStrikes = [float(x) for x in VolMaximeRaw.columns]
        strikes_to_interpolate = np.linspace((min(VolMaximeRawStrikes)), float(max(VolMaximeRawStrikes)), 100) # Création de 100 strikes avec un step régulier
        VolMaximeRaw.columns = [float(x)/spot for x in VolMaximeRaw.columns]
        VolMaximeRaw.index = [(pd.to_datetime(x, format = '%d-%b-%Y').date() - ValuationDate).days / 365 for x in VolMaximeRaw.index]

        VolMaxime = VolMaximeRaw.copy()

        for moneyness in VolMaximeRaw.columns:
            fvol = interp1d(list(VolMaximeRaw.index), list(VolMaximeRaw[moneyness].values), fill_value = 'extrapolate', kind = 'linear')
            for expiry in expiries_maxime:
                VolMaxime.at[expiry, moneyness] = fvol(expiry)

        VolMaxime = VolMaxime[VolMaxime.index.isin(expiries_maxime)].sort_index()

        MarketParamsMaxime = pd.read_csv(io.BytesIO(upload_forward_button.value[list(upload_forward_button.value.keys())[0]]['content']), skiprows=[0])
        MarketParamsMaxime = MarketParamsMaxime.reset_index(drop = True)
        MarketParamsMaxime['Term'] =  [(pd.to_datetime(x, format = '%d-%b-%Y').date() - ValuationDate).days / 365 for x in MarketParamsMaxime['Term']]
        MarketParamsMaxime['Funding rate (pts)'] = MarketParamsMaxime['Funding rate(%)'] / 100
        MarketParamsMaxime['Div & Sec Lending yield (pts)'] = MarketParamsMaxime['Div & Sec Lending yield'] / 100

        SVIParameters = []
        VolatilitySurfFilteredByExpiry = []
        expiriesinyears = list(VolMaxime.index)

        frate = interp1d(MarketParamsMaxime['Term'], MarketParamsMaxime['Funding rate (pts)'], fill_value = 'extrapolate', kind = 'linear')
        fdvd = interp1d(MarketParamsMaxime['Term'], MarketParamsMaxime['Div & Sec Lending yield (pts)'], fill_value = 'extrapolate', kind = 'linear')

        for i in tqdm(range(len(expiriesinyears))):

            status_label.value = 'Running optimisation (' + str(i) + '/' + str(len(expiriesinyears)) + ') (...)'

            time2Maturity = expiriesinyears[i]

            filtered_df = VolMaxime[VolMaxime.index == time2Maturity]

            interest = frate(time2Maturity)
            dividend = fdvd(time2Maturity)

            VolatilitySurfFilteredByExpiry.append(pd.DataFrame({'INTEREST' : interest, 'DIVIDEND' : dividend, 'Log-moneyness' : np.log(filtered_df.columns) - (interest - dividend) * time2Maturity, 'Moneyness' : filtered_df.columns, 'Strike' : filtered_df.columns * spot, 'Volatility' : filtered_df.values[0], 'Expiry' : time2Maturity}))


            args = (((VolatilitySurfFilteredByExpiry[i]['Volatility']) / 100) ** 2,
                    'Call', spot,
                    interest, dividend,
                    time2Maturity,
                    VolatilitySurfFilteredByExpiry[i]['Strike'])

            boundsSVIParameters = [(0.,100.), (0.,150.), (-0.999,0.999), (-15.,15.), (0.,5.)] # borne qu'on se fixe pour les paramètres : par exemple rho entre -1 et 1...

            global xProgress
            xProgress = []

            resultsOptimization = sciopt.differential_evolution(globalOptimization,
                                                                args = args,
                                                                callback=callbackFunction,
                                                                bounds = boundsSVIParameters,
                                                                polish=True,
                                                                maxiter=1000,
                                                                tol = 1E-6)

            SVIParameters.append(resultsOptimization.x)
            
            print(resultsOptimization)

        SVIParameters_df = pd.DataFrame(SVIParameters).rename(columns = {0 : 'a', 1 :'b', 2 : 'rho', 3 : 'm', 4 : 'sigma'})
        SVIParameters_df['Expiry'] = expiriesinyears
        SVIParameters_df.to_excel('SVI Parameters.xlsx')

        with html_out:
            display(FileLink('SVI Parameters.xlsx'))
        # download_box = widgets.VBox([html_out], align_items="center")
            
        SmoothImpliedVolSurf = pd.DataFrame({})

        for strike in strikes_to_interpolate:
            for i in (range(len(expiriesinyears))):
                
                log_mon = math.log(strike / spot) - (VolatilitySurfFilteredByExpiry[i]['INTEREST'][0] - VolatilitySurfFilteredByExpiry[i]['DIVIDEND'][0]) * (VolatilitySurfFilteredByExpiry[i]['Expiry'][0] / 365)

                ivol = SVIFunction(SVIParameters[i][0], SVIParameters[i][1], SVIParameters[i][2], SVIParameters[i][3], SVIParameters[i][4], log_mon)

                SmoothImpliedVolSurf = SmoothImpliedVolSurf.append(pd.DataFrame({'Strike' : [strike], 'Expiry' : [expiriesinyears[i]], 'Volatility' : [ivol]}))
                
        SmoothImpliedVolSurf['Moneyness'] = SmoothImpliedVolSurf['Strike'] / spot * 100
        SmoothImpliedVolSurf = SmoothImpliedVolSurf.reset_index(drop = True)
        SmoothImpliedVolSurf['Date'] = ValuationDate
        SmoothImpliedVolSurf['SPOT'] = spot
        SmoothImpliedVolSurf['ID'] = underlying_box.value
        SmoothImpliedVolSurf['Volatility'] *= 100
        
        InterestCrv = MarketParamsMaxime[['Term','Funding rate (pts)']].rename(columns = {'Term' : 'TIME', 'Funding rate (pts)' : 'RATE'})
        InterestCrv['DATE'] = ValuationDate
        InterestCrv['TIME'] *= 365
        
        DividendCrv = MarketParamsMaxime[['Term','Div & Sec Lending yield (pts)']].rename(columns = {'Term' : 'TIME', 'Div & Sec Lending yield (pts)' : 'DIVIDEND'})
        DividendCrv['DATE'] = ValuationDate
        DividendCrv['TIME'] *= 365
        
        corrected_loc_vol = LocalVolatilitySurfaceSuperDerivatives(InterestCrv, SmoothImpliedVolSurf, DividendCrv)

        status_label.value = 'Volatility surface successfully smoothed'
        button_run2.disabled = False
        
    # except Exception as e:
    #     status_label.value = str(e)
    #     button_run2.disabled = False
        
def show_vol_file_name(_):
    vol_file_name.value = list(upload_implied_vol_button.value.keys())[0]
    
def show_forward_file_name(_):
    forward_file_name.value = list(upload_forward_button.value.keys())[0]

def remove_vol_file(_):
    upload_implied_vol_button.value.clear()
    upload_implied_vol_button._counter = 0
    vol_file_name.value = ''

def remove_forward_file(_):
    upload_forward_button.value.clear()
    upload_forward_button._counter = 0
    forward_file_name.value = ''

In [37]:
interest_rate_tickers_euro = pd.read_excel('Tickers Taux OIS EUR.xlsx').set_index('ID')
interest_rate_tickers_usd = pd.read_excel('Tickers Taux SOFR USD.xlsx').set_index('ID')

def InterestCurve(UnderlyingTicker, StartDate):

    if UnderlyingTicker in ['SX5E Index', 'SX5E']:
        interest_rate_tickers = interest_rate_tickers_euro
    elif UnderlyingTicker in ['SPX Index', 'SPX']:
        interest_rate_tickers = interest_rate_tickers_usd

    # today = datetime.date.today() # Change this to fix a certain starting date
    today = StartDate # Change this to fix a certain starting date
        
    tickers = interest_rate_tickers.index
    rate = bq.data.px_last(dates = bq.func.range(today,today))
    item2 = {'RATE': rate}
    req = bql.Request(tickers,item2)
    res = bq.execute(req)
    interest_rate_data = bql.combined_df(res)
    interest_rate_data['RATE'] = interest_rate_data['RATE'] / 100 
    interest_rate_data = interest_rate_data.join(interest_rate_tickers)
    interest_rate_data['TIME'] *= 365
    return(interest_rate_data)

def DividendCurve(UnderlyingTicker, InterestCurve, StartDate):

    data_dvd = {}
    DividendCrv = pd.DataFrame(data_dvd)
    
    today = StartDate # Change this to fix a certain starting date
    # today = StartDate # Change this to fix a certain starting date
    months = [3,6,9,12]
    five_years = today.year + 5
    years = range(today.year,five_years)
    
    UnderlyingTicker = UnderlyingTicker.replace(' Index','')
    
    price = bq.data.px_last(dates = bq.func.range(today,today))
    item1 = {'PX_LAST': price}
    req = bql.Request(UnderlyingTicker + ' Index', item1)
    res = bq.execute(req)
    SAA = bql.combined_df(res)
    spot = SAA['PX_LAST'][0]
    arbitrary_strike = round_to_nearest_k_multiple(spot,50)
    
    # five_years_threshold = today + datetime.timedelta(days = 365*5)
    OptionTickersList = []
    ExpiryList = []
    
    if today >= third_friday_date(today.year,today.month) :
        if today.month == 12:
            OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(today.year + 1,1)),'%Y-%m-%d').strftime("%m/%d/%y") + ' C' + str(arbitrary_strike) + ' Index')
            OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(today.year + 1,1)),'%Y-%m-%d').strftime("%m/%d/%y") + ' P' + str(arbitrary_strike) + ' Index')
            ExpiryList.append(datetime.datetime.strptime(str(third_friday_date(today.year + 1,1)),'%Y-%m-%d').strftime("%m/%d/%y"))
        else:
            OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(today.year,today.month + 1)),'%Y-%m-%d').strftime("%m/%d/%y") + ' C' + str(arbitrary_strike) + ' Index')
            OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(today.year,today.month + 1)),'%Y-%m-%d').strftime("%m/%d/%y") + ' P' + str(arbitrary_strike) + ' Index')
            ExpiryList.append(datetime.datetime.strptime(str(third_friday_date(today.year,today.month + 1)),'%Y-%m-%d').strftime("%m/%d/%y"))
    else:
        OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(today.year,today.month)),'%Y-%m-%d').strftime("%m/%d/%y") + ' C' + str(arbitrary_strike) + ' Index')
        OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(today.year,today.month)),'%Y-%m-%d').strftime("%m/%d/%y") + ' P' + str(arbitrary_strike) + ' Index')
        ExpiryList.append(datetime.datetime.strptime(str(third_friday_date(today.year,today.month)),'%Y-%m-%d').strftime("%m/%d/%y"))

    for year in years:
        for month in months:
            if third_friday_date(year,month) > today:
                OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(year,month)),'%Y-%m-%d').strftime("%m/%d/%y") + ' C' + str(arbitrary_strike) + ' Index')
                OptionTickersList.append(UnderlyingTicker + ' ' + datetime.datetime.strptime(str(third_friday_date(year,month)),'%Y-%m-%d').strftime("%m/%d/%y") + ' P' + str(arbitrary_strike) + ' Index')
                ExpiryList.append(datetime.datetime.strptime(str(third_friday_date(year,month)),'%Y-%m-%d').strftime("%m/%d/%y"))

    mid_price = bq.data.px_mid(dates = bq.func.range(today,today)) #px_mid or px_last
    item2 = {'PRICE': mid_price}

    req = bql.Request(OptionTickersList,item2)
    res = bq.execute(req)
    call_put_prices_data = bql.combined_df(res)
    call_put_prices_data = call_put_prices_data[~call_put_prices_data['PRICE'].isna()]
        
    f = interp1d(InterestCurve['TIME'],InterestCurve['RATE'], fill_value = 'extrapolate', kind = 'linear')
    tenors = []
    implied_dividends = []
    expiries_kept = []

    call_put_prices_data = call_put_prices_data[~call_put_prices_data['PRICE'].isna()]
    for expiry in ExpiryList:
        associated_option_tickers = [index for index in call_put_prices_data.index if expiry in index]
        if sum(call_put_prices_data.index.isin(associated_option_tickers)) == 2:
        #     call_put_prices_data = call_put_prices_data.drop(associated_option_tickers)
        # else:
            option_prices_for_CP_parity = call_put_prices_data[call_put_prices_data.index.isin(associated_option_tickers)].reset_index()
            option_prices_for_CP_parity['TIME'] = (datetime.datetime.strptime(expiry,"%m/%d/%y").date() - today).days

            tenors.append(option_prices_for_CP_parity['TIME'][0])
            expiries_kept.append(datetime.datetime.strptime(expiry,"%m/%d/%y").date())

            interest_rate = float(f(option_prices_for_CP_parity['TIME'][0]))
            implied_dividend = math.log(spot/(option_prices_for_CP_parity['PRICE'][0] - option_prices_for_CP_parity['PRICE'][1] + arbitrary_strike * math.exp(- interest_rate * (option_prices_for_CP_parity['TIME'][0] / 365)))) / (option_prices_for_CP_parity['TIME'][0] / 365)
            implied_dividends.append(max(0,implied_dividend))

    dates = [today] * len(tenors)

    dvd_data = pd.DataFrame({'DATE' : dates, 'DIVIDEND' : implied_dividends, 'TIME' : tenors, 'EXPIRY' : expiries_kept})
    
    return(dvd_data)

def round_up_to_nearest_k_multiple(num,k): #To calculate implicit parameters for the call
    return math.ceil(num / k) * k
def round_down_to_nearest_k_multiple(num,k): #To calculate implicit parameters for the put
    return math.floor(num / k) * k
def round_to_nearest_k_multiple(num,k):
    if round_up_to_nearest_k_multiple(num,k) - num > num - round_down_to_nearest_k_multiple(num,k):
        return(round_down_to_nearest_k_multiple(num,k))
    else:
        return(round_up_to_nearest_k_multiple(num,k))
    
c = calendar.Calendar(firstweekday = calendar.SUNDAY)
def third_friday_date(year,month):
    monthcal = c.monthdatescalendar(year,month)
    third_friday = [day for week in monthcal for day in week if day.weekday() == calendar.FRIDAY and day.month == month][2]
    return(third_friday)

def ImpliedCallVolatility(UnderlyingPrice, ExercisePrice, Time, Interest, Target, Dividend):
    High = 5
    Low = 0
    while High - Low > 0.0001:
        if OptionEU('Call', UnderlyingPrice, ExercisePrice, Time, Interest, (High + Low) / 2, Dividend).price() > Target:
            High = (High + Low) / 2
        else: 
            Low = (High + Low) / 2
    Result = (High + Low) / 2
    return(Result)

def ImpliedPutVolatility(UnderlyingPrice, ExercisePrice, Time, Interest, Target, Dividend):
    High = 5
    Low = 0
    while High - Low > 0.0001:
        if OptionEU('Put', UnderlyingPrice, ExercisePrice, Time, Interest, (High + Low) / 2, Dividend).price() > Target:
            High = (High + Low) / 2
        else: 
            Low = (High + Low) / 2
    Result = (High + Low) / 2
    return(Result)

class OptionEU(object):
    def __init__(self, OptionType, UnderlyingPrice, ExercisePrice, Time, Interest, Volatility, Dividend):
        self.Parameters, self.OptionType, self.UnderlyingPrice, self.ExercisePrice, self.Time, self.Interest, self.Volatility, self.Dividend = (OptionType, UnderlyingPrice, ExercisePrice, Time, Interest, Volatility, Dividend), OptionType, UnderlyingPrice, ExercisePrice, Time, Interest, Volatility, Dividend
        if OptionType not in ['call','Call','CALL','put','Put','PUT']:
            raise ValueError("Erreur: type d'option invalide")
        if UnderlyingPrice < 0 or ExercisePrice < 0 or Time <= 0 or Volatility < 0:
            raise ValueError('Erreur: paramètres avec valeur(s) invalide(s)')
        d1 = (math.log(UnderlyingPrice / ExercisePrice) + (Interest - Dividend + (Volatility ** 2) / 2) * Time) / (Volatility * math.sqrt(Time))
        self.d1 = d1
        self.d2 = d1 - Volatility * math.sqrt(Time)
        
    def price(self):
        if self.OptionType in ['call','Call','CALL']:
            price = self.UnderlyingPrice * math.exp(-self.Dividend * self.Time) * norm.cdf(self.d1) - self.ExercisePrice * math.exp(-self.Interest * self.Time) * norm.cdf(self.d2)
        else:
            price = self.ExercisePrice * math.exp(-self.Interest * self.Time) * norm.cdf(-self.d2) - self.UnderlyingPrice * math.exp(-self.Dividend * self.Time) * norm.cdf(-self.d1)
        return(price)
    
    def delta(self):
        if self.OptionType in ['call','Call','CALL']:
            delta = math.exp(-self.Dividend * self.Time) * norm.cdf(self.d1) 
        else:
            delta =  math.exp(-self.Dividend * self.Time) * (norm.cdf(self.d1) - 1)
        return(delta)
    
    def gamma(self):
        gamma = math.exp(-self.Dividend * self.Time) * norm.pdf(self.d1) / (self.UnderlyingPrice * self.Volatility * math.sqrt(self.Time))
        return(gamma)

    def dollar_gamma(self): # J'ai utilisé la définition sur le chapitre 5 sur les Grecques que Maxime m'a partagé
        dollar_gamma = self.gamma() * (self.UnderlyingPrice)**2 / 100
        return(dollar_gamma)

    def theta(self): # Attention, non dailysé
#        if self.OptionType in ['call','Call','CALL']:
#            theta = - (self.UnderlyingPrice * self.Volatility * norm.pdf(self.d1) / (2 * math.sqrt(self.Time))) - (self.Interest * self.ExercisePrice * math.exp(-self.Interest * self.Time) * norm.cdf(self.d2)) #- self.Dividend * self.UnderlyingPrice * math.exp(-self.Dividend * self.Time) * norm.cdf(self.d1)))
#        else:
#            theta = - (self.UnderlyingPrice * self.Volatility * norm.pdf(self.d1) / (2 * math.sqrt(self.Time))) + (self.Interest * self.ExercisePrice * math.exp(-self.Interest * self.Time) * norm.cdf(-self.d2)) #+ self.Dividend * self.UnderlyingPrice * math.exp(-self.Dividend * self.Time) * norm.cdf(-self.d1)))
        if self.OptionType in ['call','Call','CALL']:
            theta = - (self.UnderlyingPrice * self.Volatility * math.exp(-self.Dividend * self.Time) * norm.pdf(self.d1) / (2 * math.sqrt(self.Time))) - (self.Interest * self.ExercisePrice * math.exp(-self.Interest * self.Time) * norm.cdf(self.d2)) + self.Dividend * self.UnderlyingPrice * math.exp(-self.Dividend * self.Time) * norm.cdf(self.d1)
        else:
            theta = - (self.UnderlyingPrice * self.Volatility * math.exp(-self.Dividend * self.Time) * norm.pdf(self.d1) / (2 * math.sqrt(self.Time))) + (self.Interest * self.ExercisePrice * math.exp(-self.Interest * self.Time) * norm.cdf(-self.d2)) - self.Dividend * self.UnderlyingPrice * math.exp(-self.Dividend * self.Time) * norm.cdf(-self.d1)
        return(theta)
    
    def vega(self): # Attention, non normalisé par 100
        vega = self.UnderlyingPrice * math.exp(-self.Dividend * self.Time) * math.sqrt(self.Time) * norm.pdf(self.d1)
        return(vega)
    
    def rho(self): # Attention, non normalisé par 100
        if self.OptionType in ['call','Call','CALL']: 
            rho = self.ExercisePrice * self.Time * math.exp(-self.Interest * self.Time) * norm.cdf(self.d2) 
        else:
            rho = - self.ExercisePrice * self.Time * math.exp(-self.Interest * self.Time) * norm.cdf(-self.d2)
        return(rho)
    
    def psi(self):
        if self.OptionType in ['call','Call','CALL']: 
            epsilon = - self.UnderlyingPrice * self.Time * math.exp(-self.Dividend * self.Time) * norm.cdf(self.d1)
        else:
            epsilon = self.UnderlyingPrice * self.Time * math.exp(-self.Dividend * self.Time) * norm.cdf(-self.d1)
        return(epsilon)
    
    def vanna(self):
        vanna = - math.exp(-self.Dividend * self.Time) * norm.pdf(self.d1) * self.d2 / self.Volatility
        return(vanna)
    
    def strike_greek(self):
        if self.OptionType in ['call','Call','CALL']: 
            strike_greek = - math.exp(-self.Interest * self.Time) * norm.cdf(self.d2) 
        else:
            strike_greek = math.exp(-self.Interest * self.Time) * norm.cdf(-self.d2) 
        return(strike_greek)
    
    def phi(self):
        phi = math.exp(-self.Interest * self.Time) * (1 / self.ExercisePrice) * (1 / math.sqrt(2 * math.pi * (self.Volatility ** 2) * self.Time) * math.exp(- (1 / (2 * (self.Volatility ** 2) * self.Time)) * (math.log(self.ExercisePrice / self.UnderlyingPrice) - (self.Interest - self.Dividend - (self.Volatility ** 2) / 2) * self.Time) ** 2))
        return(phi)

def VolatilitySurface(UnderlyingTicker, InterestCurve, DividendCurve, StartDate):
    # MoneynessList = [100, 102.5, 105, 107.5, 110, 112.5, 115, 117.5, 120, 80, 82.5, 85, 87.5, 90, 92.5, 95, 97.5]
    ExpiryList = list(DividendCurve['EXPIRY'])
    ExpiryListBBergFormat = [datetime.datetime.strptime(str(x),'%Y-%m-%d').strftime("%m/%d/%y") for x in ExpiryList]

    data_ivol = {}
    VolatilitySurf = pd.DataFrame(data_ivol)
    
    UnderlyingTicker = UnderlyingTicker.replace(' Index','')
    
    today = StartDate
    
    price = bq.data.px_last(dates = bq.func.range(today,today))
    item1 = {'PX_LAST': price}
    req = bql.Request(UnderlyingTicker + ' Index', item1)
    res = bq.execute(req)
    SAA = bql.combined_df(res)
    spot = SAA['PX_LAST'][0]
    length_strike = len(str(round(spot)))
    min_strike = round_down_to_nearest_k_multiple(0.75*spot,50)
    max_strike = round_up_to_nearest_k_multiple(1.25*spot,50)
    
    i = 0
    StrikeList = []
    while min_strike + i * 50 <= max_strike:
        StrikeList.append(min_strike + i * 50)
        i += 1
    
    # five_years_threshold = today + datetime.timedelta(days = 365*5)
    OptionTickersList = []
    
    for expiry in ExpiryListBBergFormat:
        for strike in StrikeList:
            OptionTickersList.append(UnderlyingTicker + ' ' + expiry + ' C' + str(strike) + ' Index')
            # OptionTickersList.append(UnderlyingTicker + ' ' + expiry + ' P' + str(strike) + ' Index')

    mid_price = bq.data.px_mid(dates = bq.func.range(today,today)) #px_mid or px_last
    item2 = {'PRICE': mid_price}  
    req = bql.Request(OptionTickersList,item2)
    res = bq.execute(req)
    call_put_prices_data = bql.combined_df(res)
    # call_put_prices_data = call_put_prices_data[~call_put_prices_data['PRICE'].isna()]

    for index in call_put_prices_data.index:
        match_expiry = re.search(r'(\d+/\d+/\d+)', index)
        call_put_prices_data.at[index,'EXPIRY'] = datetime.datetime.strptime(match_expiry.group(), '%m/%d/%y').date()
        match_strike = re.search(r'([C])(\d{' + str(length_strike) + '})', index)
        call_put_prices_data.at[index,'STRIKE'] = int(re.search(r'\d+', match_strike.group()).group())
        if 'C' in match_strike.group():
            call_put_prices_data.at[index,'TYPE'] = 'Call'
        # elif 'P' in match_strike.group():
        #     call_put_prices_data.at[index,'TYPE'] = 'Put'
        
    f_rate = interp1d(InterestCurve['TIME'],InterestCurve['RATE'], fill_value = 'extrapolate', kind = 'linear')
    f_dividend = interp1d(DividendCurve['TIME'],DividendCurve['DIVIDEND'], fill_value = 'extrapolate', kind = 'linear')
    
    missing_strikes = []
    for expiry in ExpiryList:
        missing_strikes += list(call_put_prices_data[call_put_prices_data['EXPIRY'] == expiry][call_put_prices_data['PRICE'].isnull()]['STRIKE'])
    missing_strikes = set(missing_strikes)
        
    call_put_prices_data = call_put_prices_data[~call_put_prices_data['STRIKE'].isin(missing_strikes)]
    call_put_prices_data['MONEYNESS'] = call_put_prices_data['STRIKE'] / spot * 100
    call_put_prices_data['EXPIRY IN DAYS'] = [(x - today).days for x in call_put_prices_data['EXPIRY']]
    call_put_prices_data['INTEREST'] = f_rate(call_put_prices_data['EXPIRY IN DAYS'])
    call_put_prices_data['DIVIDEND'] = f_dividend(call_put_prices_data['EXPIRY IN DAYS'])
    call_put_prices_data['SPOT'] = spot
    
    for index in call_put_prices_data.index:
        call_put_prices_data.at[index, 'IMPLIED VOLATILITY'] = ImpliedCallVolatility(spot, call_put_prices_data['STRIKE'][index], call_put_prices_data['EXPIRY IN DAYS'][index] / 365, call_put_prices_data['INTEREST'][index], call_put_prices_data['PRICE'][index], call_put_prices_data['DIVIDEND'][index]) * 100
    
    outlying_volatilies = call_put_prices_data[call_put_prices_data['IMPLIED VOLATILITY'] < 1] 
    expiries_to_remove = set(list(outlying_volatilies['EXPIRY']))
    call_put_prices_data = call_put_prices_data[~call_put_prices_data['EXPIRY'].isin(expiries_to_remove)]
    
    rename = {'EXPIRY IN DAYS' : 'Expiry', 'MONEYNESS' : 'Moneyness', 'IMPLIED VOLATILITY' : 'Volatility', 'STRIKE' : 'Strike', 'DATE' : 'Date'}
    VolatilitySurf = call_put_prices_data.rename(columns = rename).reset_index(drop = True)
    # VolatilitySurf = VolatilitySurf[list(rename.values())]
    VolatilitySurf['ID'] = UnderlyingTicker + ' Index'
    
    return(VolatilitySurf)    

In [38]:
def SmoothImpliedVolatilitySurface(VolatilitySurface):
    
    strikes_to_interpolate = np.linspace(min(VolatilitySurface['Strike']), max(VolatilitySurface['Strike']), 100) # Création de 100 strikes avec un step régulier
    spot = VolatilitySurface['SPOT'][0] # S_0

    VolatilitySurface['Log-moneyness'] = np.log(VolatilitySurface['Strike'] / spot) - (VolatilitySurface['INTEREST'] - VolatilitySurface['DIVIDEND']) * (VolatilitySurface['Expiry'] / 365)

    SVIParameters = []
    
    VolatilitySurfaceFilteredByExpiry = []
    expiriesinyears = list(set(VolatilitySurface['Expiry']/365))

    for i in tqdm(range(len(expiriesinyears))):
        
        status_label.value = 'Running optimisation (' + str(i) + '/' + str(len(expiriesinyears)) + ') (...)'

        time2Maturity = expiriesinyears[i]

        VolatilitySurfaceFilteredByExpiry.append(VolatilitySurface[VolatilitySurface['Expiry'] == round(time2Maturity * 365)].reset_index(drop = True))

        interest = VolatilitySurfaceFilteredByExpiry[i]['INTEREST'][0]
        dividend = VolatilitySurfaceFilteredByExpiry[i]['DIVIDEND'][0]

        args = (((VolatilitySurfaceFilteredByExpiry[i]['Volatility']) / 100) ** 2,
                'Call', spot,
                interest, dividend,
                time2Maturity,
                VolatilitySurfaceFilteredByExpiry[i]['Strike'])

        boundsSVIParameters = [(0.,100.), (0.,150.), (-0.999,0.999), (-15.,15.), (0.,5.)] # borne qu'on se fixe pour les paramètres : par exemple rho entre -1 et 1...

        global xProgress
        
        xProgress=[]

        resultsOptimization = sciopt.differential_evolution(globalOptimization,
                                                            args = args,
                                                            callback=callbackFunction,
                                                            bounds = boundsSVIParameters,
                                                            polish=True,
                                                            maxiter=1000,
                                                            tol = 1E-6)

        SVIParameters.append(resultsOptimization.x)
        
    SVIParameters_df = pd.DataFrame(SVIParameters).rename(columns = {0 : 'a', 1 :'b', 2 : 'rho', 3 : 'm', 4 : 'sigma'})
    SVIParameters_df['Expiry'] = expiriesinyears
    SVIParameters_df.to_excel('SVI Parameters.xlsx')

    with html_out:
        display(FileLink('SVI Parameters.xlsx'))
    # download_box = widgets.VBox([html_out], align_items="center")
        
    SmoothImpliedVolSurf = pd.DataFrame({})

    for strike in strikes_to_interpolate:
        for i in (range(len(expiriesinyears))):

            log_mon = math.log(strike / spot) - (VolatilitySurfaceFilteredByExpiry[i]['INTEREST'][0] - VolatilitySurfaceFilteredByExpiry[i]['DIVIDEND'][0]) * (VolatilitySurfaceFilteredByExpiry[i]['Expiry'][0] / 365)

            ivol = SVIFunction(SVIParameters[i][0], SVIParameters[i][1], SVIParameters[i][2], SVIParameters[i][3], SVIParameters[i][4], log_mon)

            SmoothImpliedVolSurf = SmoothImpliedVolSurf.append(pd.DataFrame({'Strike' : [strike], 'Expiry' : [expiriesinyears[i]], 'Volatility' : [ivol]}))

    expiries_to_interpolate = np.linspace(min(SmoothImpliedVolSurf['Expiry']), max(SmoothImpliedVolSurf['Expiry']), 17)

    SmoothImpliedVolSurfFinal = pd.DataFrame({})

    for expiry in expiries_to_interpolate:
        if expiry not in SmoothImpliedVolSurf['Expiry']:
            for strike in strikes_to_interpolate:

                filtered_surf = SmoothImpliedVolSurf[SmoothImpliedVolSurf['Strike'] == strike]
                f_vol = interp1d(filtered_surf['Expiry'], filtered_surf['Volatility'], fill_value = 'extrapolate', kind = 'linear')

                SmoothImpliedVolSurfFinal = SmoothImpliedVolSurfFinal.append(pd.DataFrame({'Strike' : [strike], 'Expiry' : [expiry], 'Volatility' : [float(f_vol(expiry))]}))

    SmoothImpliedVolSurfFinal['Moneyness'] = SmoothImpliedVolSurfFinal['Strike'] / spot * 100
    SmoothImpliedVolSurfFinal = SmoothImpliedVolSurfFinal.reset_index(drop = True)
    SmoothImpliedVolSurfFinal['Date'] = VolatilitySurface['Date'][0]
    SmoothImpliedVolSurfFinal['SPOT'] = VolatilitySurface['SPOT'][0]
    SmoothImpliedVolSurfFinal['ID'] = VolatilitySurface['ID'][0]
    SmoothImpliedVolSurfFinal['Volatility'] *= 100
    
    return(SmoothImpliedVolSurfFinal)


def LocalVolatilitySurface(InterestCurve, VolatilitySurface, DividendCurve):
    
    spot = VolatilitySurface['SPOT'][0] # S_0
    strikes_to_interpolate = np.linspace(min(VolatilitySurface['Strike']), max(VolatilitySurface['Strike']), 100) # Création de 100 strikes avec un step régulier
    expiries_to_interpolate = np.linspace(min(VolatilitySurface['Expiry']), max(VolatilitySurface['Expiry']), 17)

    ivol_matrix = (VolatilitySurface.pivot_table(columns='Strike', index='Expiry', values='Volatility').values) / 100
    
    dvd_interpolation = interp1d(DividendCurve['TIME'] / 365, DividendCurve['DIVIDEND'], fill_value = 'extrapolate')
    rate_interpolation = interp1d(InterestCurve['TIME'] / 365, InterestCurve['RATE'], fill_value = 'extrapolate')

    mons_to_interpolate = [x/spot for x in strikes_to_interpolate]

    loc_vol_df = pd.DataFrame({})

    for i in tqdm(range(1, len(mons_to_interpolate) - 1)):
        for j in range(1, len(expiries_to_interpolate)):

            ivol_t_k = ivol_matrix[j][i]
            ivol_tminus_k = ivol_matrix[j-1][i]
            ivol_t_kminus = ivol_matrix[j][i-1]
            ivol_t_kplus = ivol_matrix[j][i+1]

            rate_t = float(rate_interpolation(expiries_to_interpolate[j]))
            dvd_t = float(dvd_interpolation(expiries_to_interpolate[j]))

            d1 = (math.log(1/mons_to_interpolate[i]) + (rate_t - dvd_t + (ivol_t_k**2) / 2) * expiries_to_interpolate[j]) / (ivol_t_k * math.sqrt(expiries_to_interpolate[j]))

            dsigma_dt = (ivol_t_k - ivol_tminus_k) / ((expiries_to_interpolate[j] - expiries_to_interpolate[j-1]))
            dsigma_dk = (ivol_t_k - ivol_t_kminus) / ((mons_to_interpolate[i] - mons_to_interpolate[i-1]))
            d2sigma_dk2 = (ivol_t_kplus - 2 * ivol_t_k + ivol_t_kminus) / ((mons_to_interpolate[i] - mons_to_interpolate[i-1])**2)

            numerator = ivol_t_k ** 2 + 2 * ivol_t_k * expiries_to_interpolate[j] * dsigma_dt + 2 * (rate_t - dvd_t) * ivol_t_k * mons_to_interpolate[i] * expiries_to_interpolate[j] * dsigma_dk
            denominator = ((1 + mons_to_interpolate[i] * d1 * dsigma_dk * math.sqrt(expiries_to_interpolate[j])) ** 2) + ivol_t_k * (mons_to_interpolate[i]**2) * expiries_to_interpolate[j] * (d2sigma_dk2 - d1 * (dsigma_dk**2) * math.sqrt(expiries_to_interpolate[j]))

            loc_vol = math.sqrt(max(0,numerator / denominator))

            if loc_vol == 0:
                loc_vol = (ivol_t_k)
            # print(loc_vol)

            data = pd.DataFrame({'Expiry' : [expiries_to_interpolate[j] * 365], 'Moneyness' : [mons_to_interpolate[i] * 100], 'Local Volatility' : [loc_vol * 100], 'Expiry in years' : [expiries_to_interpolate[j]], 'Strike' : [mons_to_interpolate[i] * spot]})

            loc_vol_df = loc_vol_df.append(data, ignore_index = True)

    loc_vol_df = loc_vol_df.rename(columns = {'Local Volatility' : 'Volatility'})
    
    return(loc_vol_df)

def CorrectLocalVolatilitySurface(LocalVolatilitySurface):
    
    threshold = np.percentile(LocalVolatilitySurface['Volatility'], 5) - 2.5
    unproblematic_loc_vols = LocalVolatilitySurface[LocalVolatilitySurface['Volatility'] > threshold]
    problematic_loc_vols = LocalVolatilitySurface[LocalVolatilitySurface['Volatility'] <= threshold]

    for index in problematic_loc_vols.index:
        correct_loc_vol = interp1d(unproblematic_loc_vols[unproblematic_loc_vols['Moneyness'] == problematic_loc_vols['Moneyness'][index]]['Expiry'], unproblematic_loc_vols[unproblematic_loc_vols['Moneyness'] == problematic_loc_vols['Moneyness'][index]]['Volatility'], fill_value = 'extrapolate')
        problematic_loc_vols['Volatility'][index] = correct_loc_vol(problematic_loc_vols['Expiry'][index])

    corrected_loc_vol = problematic_loc_vols.append(unproblematic_loc_vols).sort_index()
    
    return(corrected_loc_vol)

In [39]:
def runSVI_Bloomberg(_):

    # try:
    
        status_label.value = 'Retrieving Bloomberg data'
    
        UnderlyingTicker = underlying_box.value
        UnderlyingTicker = UnderlyingTicker.replace(' Index','')
        today = valuation_date.value
        
        global VolatilitySurf, VolatilitySurfSmooth, corrected_loc_vol, InterestCrv, DividendCrv, spot
        
        InterestCrv = InterestCurve(UnderlyingTicker, today)
        DividendCrv = DividendCurve(UnderlyingTicker, InterestCrv, today)
        VolatilitySurf = VolatilitySurface(UnderlyingTicker, InterestCrv, DividendCrv, today)
        spot = VolatilitySurf['SPOT'][0]
        
        VolatilitySurfSmooth = SmoothImpliedVolatilitySurface(VolatilitySurf) 
        loc_vol_df = LocalVolatilitySurface(InterestCrv, VolatilitySurfSmooth, DividendCrv)
        corrected_loc_vol = CorrectLocalVolatilitySurface(loc_vol_df)

        status_label.value = 'Volatility surface successfully smoothed'
        button_run2.disabled = False
        
        return(corrected_loc_vol)
        
    # except Exception as e:
    #     status_label.value = str(e)
    #     button_run2.disabled = False

In [40]:
def ForwardInterestRateBis(InterestCurve, t, T):
    
    f_rate = interp1d(InterestCurve['TIME'] / 365, InterestCurve['RATE'], fill_value = 'extrapolate')
    rate1 = float(f_rate(t))
    rate2 = float(f_rate(T))
    
    if t != 0:
        forward_rate = ((((1+rate2) ** T)) / ((1+rate1) ** t)) ** (1 / (T - t)) - 1
        return(forward_rate)
    else:
        return(rate1)

def ForwardDividendBis(DividendCurve, t, T):
    
    f_dvd = interp1d(DividendCurve['TIME'] / 365, DividendCurve['DIVIDEND'], fill_value = 'extrapolate', kind = 'linear')
    dvd1 = float(f_dvd(t))
    dvd2 = float(f_dvd(T))
    
    if t != 0:
        forward_dvd= ((((1+dvd2) ** T) / ((1+dvd1) ** t)) ** (1 / (T - t))) - 1
        return(forward_dvd)
    else:
        return(dvd1)

In [41]:
def match_vanillas(_):
    
    ValuationDate = valuation_date.value
    UnderlyingTicker = underlying_box.value.replace(' Index','')

    months = [3,6,9,12]
    three_years = ValuationDate.year + 3
    years = range(ValuationDate.year,three_years)
    EndDates = []
    
    if ValuationDate >= third_friday_date(ValuationDate.year,ValuationDate.month) :
        if ValuationDate.month == 12:
            EndDates.append(third_friday_date(ValuationDate.year + 1,1))
        else:
            EndDates.append(third_friday_date(ValuationDate.year,ValuationDate.month + 1))
    else:
        EndDates.append(third_friday_date(ValuationDate.year,ValuationDate.month))

    for year in years:
        for month in months:
            if third_friday_date(year,month) > ValuationDate:
                EndDates.append(third_friday_date(year,month))
    
    EndDates =  [datetime.date(2023, 3, 17), datetime.date(2023, 6, 16), datetime.date(2023, 9, 15), datetime.date(2023, 12, 15), datetime.date(2024, 3, 15), datetime.date(2024, 6, 21), datetime.date(2024, 9, 20), datetime.date(2024, 12, 20)] # Expiries pour lesquelles on souhaite calculer les prix d'options
    lower_strike = round_down_to_nearest_k_multiple(spot * 0.6,100)
    upper_strike = round_up_to_nearest_k_multiple(spot * 1.4,100)
    strikes = np.linspace(lower_strike, upper_strike, round((upper_strike - lower_strike) / 100 + 1))        
    
    call_errors = pd.DataFrame()
    put_errors = pd.DataFrame()
    NumberPathsByExpiries = number_MC_paths.value

    for EndDate in (EndDates):
        
        status_label2.value = 'Calculating expiry : ' + str(EndDate)

        T = (EndDate - ValuationDate).days / 365

        MC = MC_localvol_allstrikes(ValuationDate, EndDate, spot, InterestCrv, DividendCrv, corrected_loc_vol, NumberPathsByExpiries)

        premium_call = MC['Call Premium']
        premium_put = MC['Put Premium']

        length_strike = len(str(round(spot))) # Taille du string des strikes
        EndDateBBergFormat = datetime.datetime.strptime(str(EndDate),'%Y-%m-%d').strftime("%m/%d/%y")

        list_call_tickers = [UnderlyingTicker + ' ' + (EndDateBBergFormat) + ' C' + str(x) + ' Index' for x in strikes]
        list_put_tickers = [UnderlyingTicker + ' ' + (EndDateBBergFormat) + ' P' + str(x) + ' Index' for x in strikes]

        mid_price = bq.data.px_mid(dates = bq.func.range(ValuationDate,ValuationDate)) #px_mid or px_last
        item2 = {'PRICE': mid_price}  

        req = bql.Request(list_call_tickers,item2)
        res = bq.execute(req)
        call_prices_data = bql.combined_df(res)

        req = bql.Request(list_put_tickers,item2)
        res = bq.execute(req)
        put_prices_data = bql.combined_df(res)

        for index in call_prices_data.index:
                match_expiry = re.search(r'(\d+/\d+/\d+)', index)
                call_prices_data.at[index,'EXPIRY'] = datetime.datetime.strptime(match_expiry.group(), '%m/%d/%y').date()
                match_strike = re.search(r'([C])(\d{' + str(length_strike) + '})', index)
                call_prices_data.at[index,'STRIKE'] = int(re.search(r'\d+', match_strike.group()).group())
                call_prices_data.at[index,'TYPE'] = 'Call'
        call_prices_data = call_prices_data.reset_index().rename(columns = {'ID' : 'Ticker'})

        for index in put_prices_data.index:
                match_expiry = re.search(r'(\d+/\d+/\d+)', index)
                put_prices_data.at[index,'EXPIRY'] = datetime.datetime.strptime(match_expiry.group(), '%m/%d/%y').date()
                match_strike = re.search(r'([P])(\d{' + str(length_strike) + '})', index)
                put_prices_data.at[index,'STRIKE'] = int(re.search(r'\d+', match_strike.group()).group())
                put_prices_data.at[index,'TYPE'] = 'Put'
        put_prices_data = put_prices_data.reset_index().rename(columns = {'ID' : 'Ticker'})

        call_prices_data['Local Vol Prices'] = premium_call
        put_prices_data['Local Vol Prices'] = premium_put

        call_prices_data['Error in bps'] = abs((call_prices_data['PRICE'] - call_prices_data['Local Vol Prices']) / spot / T * 10000)
        put_prices_data['Error in bps'] = abs((put_prices_data['PRICE'] - put_prices_data['Local Vol Prices']) / spot / T * 10000)

        call_errors = call_errors.append(call_prices_data)
        put_errors = put_errors.append(put_prices_data)
        
    call_errors = call_errors[call_errors['STRIKE'] > round_down_to_nearest_k_multiple(spot, 100)]
    put_errors = put_errors[put_errors['STRIKE'] < round_up_to_nearest_k_multiple(spot, 100)]
        
    call_errors_matrix = call_errors.pivot_table(columns='STRIKE', index='EXPIRY', values='Error in bps')

    z = call_errors_matrix.values
    where_are_NaNs = np.isnan(z)
    z[where_are_NaNs] = 1000
    x = [a / spot * 100 for a in call_errors_matrix.columns]
    y = call_errors_matrix.index
    y = [(a - ValuationDate).days / 365 for a in y]
    annotations = go.Annotations()
    for n, row in enumerate(z):
        for m, val in enumerate(row):
            annotations.append(go.Annotation(text=str(round(z[n][m])), x=x[m], y=y[n],
                                                 xref='x1', yref='y1', showarrow=False))
    layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        title = dict(text = '<b>Call price matching errors in bps(' + str(NumberPathsByExpiries) + ' paths by expiries) </b>', xanchor = 'center', font = dict(color = '#ffa500')),
        font_color = 'white',
        annotations = annotations,
        title_x = 0.5,        
        margin=dict(t=30),
        xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Strike (%)',)),
        yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Expiry (years)')))
        
    call_errors_matrix_grid = go.FigureWidget(go.Heatmap(z = z, x = x, y = y, text = z,  colorscale = 'geyser'), layout = layout) 
    call_errors_matrix_grid.update_yaxes(autorange="reversed")
        
    put_errors_matrix = put_errors.pivot_table(columns='STRIKE', index='EXPIRY', values='Error in bps')

    z = put_errors_matrix.values
    where_are_NaNs = np.isnan(z)
    z[where_are_NaNs] = 1000
    x = [a / spot * 100 for a in put_errors_matrix.columns]
    y = put_errors_matrix.index
    y = [(a - ValuationDate).days / 365 for a in y]
    annotations = go.Annotations()
    for n, row in enumerate(z):
        for m, val in enumerate(row):
            annotations.append(go.Annotation(text=str(round(z[n][m])), x=x[m], y=y[n],
                                                 xref='x1', yref='y1', showarrow=False))
    layout = go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        title = dict(text = '<b>Put price matching errors in bps(' + str(NumberPathsByExpiries) + ' paths by expiries) </b>', xanchor = 'center', font = dict(color = '#ffa500')),
        font_color = 'white',
        annotations = annotations,
        title_x = 0.5,        
        margin=dict(t=30),
        xaxis=go.layout.XAxis(title=go.layout.xaxis.Title(text='Strike (%)',)),
        yaxis=go.layout.YAxis(title=go.layout.yaxis.Title(text='Expiry (years)')))
        
    put_errors_matrix_grid = go.FigureWidget(go.Heatmap(z = z, x = x, y = y, text = z,  colorscale = 'geyser'), layout = layout) 
    put_errors_matrix_grid.update_yaxes(autorange="reversed")
    
    status_label2.value = 'Errors calculated successfully'
        
    app.children = [title1, data, title2, number_MC_paths, button_run8, status_label2, call_errors_matrix_grid, put_errors_matrix_grid]  

In [42]:
def MC_localvol_allstrikes(StartDate, EndDate, UnderlyingPrice, InterestCurve, DividendCurve, LocalVolatilitySurface, N):
    
    T = (EndDate - StartDate).days / 365
    
    lower_strike = round_down_to_nearest_k_multiple(UnderlyingPrice * 0.6,100)
    upper_strike = round_up_to_nearest_k_multiple(UnderlyingPrice * 1.4,100)
    strikes = np.linspace(lower_strike, upper_strike, round((upper_strike - lower_strike) / 100 + 1))    
    
    call_premiums = {}
    put_premiums = {}
    
    frate = interp1d(InterestCurve['TIME']/365, InterestCurve['RATE'], fill_value = 'extrapolate')
    fdvd = interp1d(DividendCurve['TIME']/365, DividendCurve['DIVIDEND'], fill_value = 'extrapolate')
    # fvol = interp2d(LocalVolatilitySurface['Expiry in years'], LocalVolatilitySurface['Moneyness'] / 100, LocalVolatilitySurface['Volatility']/100)
    
    # r = frate(T)
    # q = fdvd(T)

    z = LocalVolatilitySurface.pivot_table(columns='Moneyness', index='Expiry in years', values='Volatility').values / 100
    x = LocalVolatilitySurface.pivot_table(columns='Moneyness', index='Expiry in years', values='Volatility').index 
    y = LocalVolatilitySurface.pivot_table(columns='Moneyness', index='Expiry in years', values='Volatility').columns / 100
    fvol = interp2d(x = x, y = y, z = z.T, kind = 'linear')
        
    for X in strikes :
        call_premiums[X] = 0
        put_premiums[X] = 0
    
    df = pd.DataFrame()
    
    for i in tqdm(range(0, N)):
        
        call_payoffs = []
        put_payoffs = []
    
        St = UnderlyingPrice
        step = 0
        dt = T / (52)
        # dt = T / math.sqrt(N)
        
        # iteration = 0
        # n_iter = math.ceil(T / dt)
        # e1 = np.random.randn(n_iter)
        # e1 = np.random.standard_normal(size = n_iter)
        
        while(step < T):
            
            e1 = np.random.randn()
            
            sigma = fvol(step, St/UnderlyingPrice)
            
            r = ForwardInterestRateBis(InterestCurve, step, step + dt)
            q = ForwardDividendBis(DividendCurve, step, step + dt)
        
            St = St * math.exp((r - q - (sigma ** 2) / 2) * dt + sigma * math.sqrt(dt) * e1)
            
            # iteration += 1
            
            step += dt
            
        r = frate(T)
                        
        for X in strikes:
            
            if St > X:
                call_premiums[X] += (St - X) * np.exp(-r * T)
            if St < X:
                put_premiums[X] += (X - St) * np.exp(-r * T)

    for X in strikes:
        df = df.append(pd.DataFrame({'Strike' : [X], 'Call Premium' : [(call_premiums[X])/N], 'Put Premium' : [(put_premiums[X])/N]}))
        
    # Outer loop runs a new simulation -->
    
    df = df.reset_index(drop = True)
    
    return(df)

In [43]:
button_run2.on_click(runSVI_SuperDerivatives)
button_run3.on_click(remove_vol_file)
button_run4.on_click(remove_forward_file)
button_run5.on_click(show_vol_file_name)
button_run6.on_click(show_forward_file_name)
button_run7.on_click(runSVI_Bloomberg)
button_run8.on_click(match_vanillas)

In [44]:
title1 = widgets.HTML(value = '<b><p style="text-align:center ; font-size:20px">Step 1 : Fetching Data and Smoothing Implied Volatility</p></b>', placeholder = '', description = '', layout = widgets.Layout(height='50px'))
data_tab_1 = widgets.VBox([underlying_box, widgets.HBox([upload_implied_vol_button, button_run3, button_run5, vol_file_name]), widgets.HBox([upload_forward_button, button_run4, button_run6, forward_file_name]), valuation_date, button_run2, status_label, html_out])
data_tab_2 = widgets.VBox([underlying_box, valuation_date, button_run7, status_label, html_out])
data = widgets.Tab([data_tab_1, data_tab_2])
data.set_title(0, 'With SuperDerivatives')
data.set_title(1, 'With Bloomberg')

title2 = widgets.HTML(value = '<b><p style="text-align:center ; font-size:20px">Step 2 : Calculating Local Volatility and Pricing Errors</p></b>', placeholder = '', description = '', layout = widgets.Layout(height='50px'))


app = widgets.VBox([title1, data, title2, number_MC_paths, button_run8, status_label2])

In [45]:
app